In [1]:
import sys
import os
# Manually set the path relative to the py file's location that you want to import
func_lib_path = os.path.abspath(os.path.join(os.getcwd(),'../'))

# Add the path to sys.path
sys.path.append(func_lib_path)

# Now you can import func_lib
import func_lib
import pandas as pd

In [2]:
historical_prices = func_lib.create_hist_prices('2000-01-01', '2024-12-31')
historical_prices.head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WST,WTW,WY,WYNN,XEL,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,43.382854,0.843076,NaN,NaN,8.288179,1.277778,NaN,16.274672,28.095688,6.307347,...,5.421762,NaN,11.352946,NaN,6.848378,NaN,4.634289,NaN,25.027779,NaN
2000-01-04,40.068867,0.771997,NaN,NaN,8.051380,1.270833,NaN,14.909398,26.674353,6.241645,...,5.477769,NaN,10.926454,NaN,7.006070,NaN,4.541137,NaN,24.666668,NaN
2000-01-05,37.583401,0.783294,NaN,NaN,8.036577,1.388889,NaN,15.204173,27.063761,6.143089,...,5.455366,NaN,11.505272,NaN,7.276401,NaN,4.564423,NaN,25.138889,NaN
2000-01-06,36.152367,0.715509,NaN,NaN,8.317779,1.375000,NaN,15.328290,26.323893,6.175943,...,5.399357,NaN,12.043464,NaN,7.208818,NaN,4.525609,NaN,23.777779,NaN
2000-01-07,39.165089,0.749402,NaN,NaN,8.406583,1.451389,NaN,16.072981,27.063761,6.274495,...,5.455366,NaN,11.647435,NaN,7.208818,NaN,4.424696,NaN,23.513889,NaN


In [3]:
list_of_momentums = [1,2]

In [5]:
# Initialize the forecast horizon
forecast_horizon = 1

# Compute forward returns by taking percentage change of close prices
f_returns = historical_prices.pct_change(forecast_horizon)

In [6]:
f_returns[['AAPL']].head()

Ticker,AAPL
Date,
2000-01-03,NaN
2000-01-04,-0.084310
2000-01-05,0.014633
2000-01-06,-0.086538
2000-01-07,0.047369


In [7]:
# We then shift the forward returns
f_returns = f_returns.shift(-forecast_horizon)
f_returns.iloc[:,0:10].head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM
Date,,,,,,,,,,
2000-01-03,-0.076389,-0.084310,NaN,NaN,-0.028571,-0.005435,NaN,-0.083889,-0.050589,-0.010417
2000-01-04,-0.062030,0.014633,NaN,NaN,-0.001839,0.092897,NaN,0.019771,0.014599,-0.015790
2000-01-05,-0.038076,-0.086538,NaN,NaN,0.034990,-0.010000,NaN,0.008163,-0.027338,0.005348
2000-01-06,0.083334,0.047369,NaN,NaN,0.010676,0.055556,NaN,0.048583,0.028106,0.015957
2000-01-07,0.060576,-0.017588,NaN,NaN,-0.007043,0.033493,NaN,0.038610,0.087050,0.000000


In [8]:
# Pivot the dataframe
f_returns = pd.DataFrame(f_returns.unstack())

# Name the column based on the forecast horizon
name = "F_" + str(forecast_horizon) + "_d_returns"

f_returns.rename(columns={0: name}, inplace=True)
f_returns.head()

F_1_d_returns
Ticker Date                     
A      2000-01-03      -0.076389
       2000-01-04      -0.062030
       2000-01-05      -0.038076
       2000-01-06       0.083334
       2000-01-07       0.060576

In [9]:
# Initialize total_returns with forward returns
total_returns = f_returns

In [10]:
# Iterate over the list of momentum values
for i in list_of_momentums:
    # Compute returns for each momentum value
    feature = historical_prices.pct_change(i)
    feature = pd.DataFrame(feature.unstack())

    # Name the colum based on the momentum value
    name = str(i) + "_d_returns"
    feature.rename(columns={0:name}, inplace=True)

    # Rename columns and reset index
    feature.rename(columns={0: name, 'level_0': 'Ticker'}, inplace=True)

    # Merge computed feature returns with total_returns based on Ticker and Date
    total_returns = pd.merge(total_returns, feature, left_index=True, right_index=True, how='outer')

In [11]:
# Drop rows with any NaN values
total_returns.dropna(axis=0, how='any', inplace=True)
total_returns.head()

F_1_d_returns  1_d_returns  2_d_returns
Ticker Date                                               
A      2000-01-05      -0.038076    -0.062030    -0.133681
       2000-01-06       0.083334    -0.038076    -0.097744
       2000-01-07       0.060576     0.083334     0.042085
       2000-01-10      -0.013599     0.060576     0.148958
       2000-01-11      -0.020220    -0.013599     0.046153

In [4]:
total_return = func_lib.computingReturns(historical_prices, list_of_momentums)
total_return.head()

F_1_d_returns  1_d_returns  2_d_returns
Ticker Date                                               
A      2000-01-05      -0.038076    -0.062030    -0.133681
       2000-01-06       0.083334    -0.038076    -0.097744
       2000-01-07       0.060576     0.083334     0.042085
       2000-01-10      -0.013599     0.060576     0.148958
       2000-01-11      -0.020220    -0.013599     0.046153